In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd

import scipy.io as sio
import librosa

import time

import skimage
import skimage.io
from skimage.transform import rescale

from scipy.io import wavfile
import os
import subprocess

import sys
sys.path.append("../../src")
from spectrogramtools import *
from stego import *
from swpowerstego import *
from imtools import *
from tsp import *
from curvature import *


In [ ]:
# Image parameters
thresh = 0.5
p = 1
canny_sigma = 2.5
density_fac= 1
edge_weight = 5
target_points = 2000

np.random.seed(0)
# 31, 67
I = skimage.io.imread("../../data/caltech-101/revolver/image_0065.jpg")

X = voronoi_stipple(I, thresh, target_points=target_points, p=p, canny_sigma=canny_sigma, edge_weight=edge_weight, n_iters=10, do_plot=False)
plt.figure(figsize=(10, 10))
plt.scatter(X[:, 0], X[:, 1])
plt.axis("equal")

In [ ]:
#X = density_filter(X, fac=density_fac, k=2)
X = get_tsp_tour(X)

plt.figure(figsize=(16, 8))
plt.subplot(121)
plt.plot(X[:, 0], X[:, 1])
plt.axis("equal")
plt.title("Before resampling/smoothing")

s = get_arclen(get_curv_vectors(X, 0, 1, loop=True)[1])
X = arclen_resample(X, s, X.shape[0])
sigma = 0.2
X = get_curv_vectors(X, 0, sigma, loop=True)[0]
plt.subplot(122)
plt.plot(X[:, 0], X[: ,1])
plt.axis("equal")
plt.title("After resampling/smoothing")

## Load in audio and compute best target

In [ ]:
# Audio parameters
x, sr = librosa.load("../../data/tunes/janie.mp3", sr=44100)
win_length = 1024
mag_idxs = [1, 2]
phase_idxs = mag_idxs
print([int(fi*sr/win_length) for fi in mag_idxs])

# Sliding window parameters
win = 16
fit_lam = 0.1
q = -1

sp = STFTPowerDisjoint(x, X, win_length, mag_idxs, phase_idxs, win, fit_lam, q, do_viterbi=True)
sp.plot(normalize=False)

sp.solve()
sp.plot(normalize=True)

In [ ]:
y = sp.reconstruct_signal()
ipd.Audio(y, rate=sr)

In [ ]:
prefix = "aerosmith-revolver-2d"
filename = "{}.mp3".format(prefix)
wavfile.write("out.wav", sr, y)
if os.path.exists(filename):
    os.remove(filename)
subprocess.call("ffmpeg -i out.wav".split() + [filename], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT)
os.remove("out.wav")

z, sr = librosa.load(filename, sr=sr)
z_sp = STFTPowerDisjoint(z, X, win_length, mag_idxs, phase_idxs, win, fit_lam, q, do_viterbi=False)
z_sp.MagSolver.targets = sp.MagSolver.targets
snr = get_snr(x, z)
distortion = z_sp.get_distortion()[0]
ratio = snr/distortion

title = "snr = {:.3f}, distortion = {:.3f}".format(snr, distortion,)

plt.figure(figsize=(6, 6))
z_sp.plot(True, title)
plt.tight_layout()
plt.savefig("{}.png".format(prefix), facecolor='white')
